In [0]:
!pip install kagglehub
dbutils.library.restartPython()

In [0]:
# Define parameters
dbutils.widgets.text("CATALOG_NAME", "pedroz_catalog", "Catalog Name")
dbutils.widgets.text("SCHEMA_NAME", "metric_views_schema"s, "Schema Name")
CATALOG_NAME = dbutils.widgets.get("CATALOG_NAME")
SCHEMA_NAME = dbutils.widgets.get("SCHEMA_NAME")

In [0]:
import os

# You need to define this variable in order to download the Kaggle files to your Volume
os.environ["KAGGLEHUB_CACHE"] = f"/Volumes/pedroz_catalog/metric_views_schema/dimensional_model/kaggle_files"

In [0]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shrinivasv/retail-store-star-schema-dataset")

print("Path to dataset files:", path)

In [0]:
import glob

csv_folder = path  # path variable from kagglehub.dataset_download
csv_files = glob.glob(f"{csv_folder}/*.csv")

for csv_file in csv_files:
    table_name = os.path.splitext(os.path.basename(csv_file))[0]
    df = spark.read.csv(csv_file, header=True, inferSchema=True)
    # Clean column names: replace spaces with underscores
    for c in df.columns:
        df = df.withColumnRenamed(c, c.replace(" ", "_"))
    full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"
    spark.sql(f"DROP TABLE IF EXISTS {full_table_name}")
    df.write.saveAsTable(full_table_name)

In [0]:
display(spark.sql(f"SHOW TABLES IN {CATALOG_NAME}.{SCHEMA_NAME}"))

In [0]:
%sql
SELECT *
FROM $CATALOG_NAME.$SCHEMA_NAME.dim_campaigns
LIMIT 10